In [1]:
import os
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import torchvision.utils as vutils

from IPython.display import HTML
import torchvision.datasets as dset

In [2]:
import tensorflow as tf

import models
import models.research
import models.research.slim
import models.research.slim.nets as nets
from cleverhans.model_zoo.resNet import ResNet
from cleverhans.model_zoo.alexNet import AlexNet

from torch.utils.data.sampler import SubsetRandomSampler


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import models.research.slim.nets.resnet_utils as resnet_utils


resnet_arg_scope = resnet_utils.resnet_arg_scope
slim = tf.contrib.slim


class NoOpScope(object):
  """No-op context manager."""

  def __enter__(self):
    return None

  def __exit__(self, exc_type, exc_value, traceback):
    return False


@slim.add_arg_scope
def bottleneck(inputs,
               depth,
               depth_bottleneck,
               stride,
               rate=1,
               outputs_collections=None,
               scope=None,
               use_bounded_activations=False):
  """Bottleneck residual unit variant with BN after convolutions.
  This is the original residual unit proposed in [1]. See Fig. 1(a) of [2] for
  its definition. Note that we use here the bottleneck variant which has an
  extra bottleneck layer.
  When putting together two consecutive ResNet blocks that use this unit, one
  should use stride = 2 in the last unit of the first block.
  Args:
    inputs: A tensor of size [batch, height, width, channels].
    depth: The depth of the ResNet unit output.
    depth_bottleneck: The depth of the bottleneck layers.
    stride: The ResNet unit's stride. Determines the amount of downsampling of
      the units output compared to its input.
    rate: An integer, rate for atrous convolution.
    outputs_collections: Collection to add the ResNet unit output.
    scope: Optional variable_scope.
    use_bounded_activations: Whether or not to use bounded activations. Bounded
      activations better lend themselves to quantized inference.
  Returns:
    The ResNet unit's output.
  """
  with tf.variable_scope(scope, 'bottleneck_v1', [inputs]) as sc:
    depth_in = slim.utils.last_dimension(inputs.get_shape(), min_rank=4)
    if depth == depth_in:
      shortcut = resnet_utils.subsample(inputs, stride, 'shortcut')
    else:
      shortcut = slim.conv2d(
          inputs,
          depth, [1, 1],
          stride=stride,
          activation_fn=tf.nn.relu6 if use_bounded_activations else None,
          scope='shortcut')

    residual = slim.conv2d(inputs, depth_bottleneck, [1, 1], stride=1,
                           scope='conv1')
    residual = resnet_utils.conv2d_same(residual, depth_bottleneck, 3, stride,
                                        rate=rate, scope='conv2')
    residual = slim.conv2d(residual, depth, [1, 1], stride=1,
                           activation_fn=None, scope='conv3')

    if use_bounded_activations:
      # Use clip_by_value to simulate bandpass activation.
      residual = tf.clip_by_value(residual, -6.0, 6.0)
      output = tf.nn.relu6(shortcut + residual)
    else:
      output = tf.nn.relu(shortcut + residual)

    return slim.utils.collect_named_outputs(outputs_collections,
                                            sc.name,
                                            output)


def resnet_v1(inputs,
              blocks,
              num_classes=None,
              is_training=True,
              global_pool=True,
              output_stride=None,
              include_root_block=True,
              spatial_squeeze=True,
              store_non_strided_activations=False,
              reuse=None,
              scope=None):
  """Generator for v1 ResNet models.
  This function generates a family of ResNet v1 models. See the resnet_v1_*()
  methods for specific model instantiations, obtained by selecting different
  block instantiations that produce ResNets of various depths.
  Training for image classification on Imagenet is usually done with [224, 224]
  inputs, resulting in [7, 7] feature maps at the output of the last ResNet
  block for the ResNets defined in [1] that have nominal stride equal to 32.
  However, for dense prediction tasks we advise that one uses inputs with
  spatial dimensions that are multiples of 32 plus 1, e.g., [321, 321]. In
  this case the feature maps at the ResNet output will have spatial shape
  [(height - 1) / output_stride + 1, (width - 1) / output_stride + 1]
  and corners exactly aligned with the input image corners, which greatly
  facilitates alignment of the features to the image. Using as input [225, 225]
  images results in [8, 8] feature maps at the output of the last ResNet block.
  For dense prediction tasks, the ResNet needs to run in fully-convolutional
  (FCN) mode and global_pool needs to be set to False. The ResNets in [1, 2] all
  have nominal stride equal to 32 and a good choice in FCN mode is to use
  output_stride=16 in order to increase the density of the computed features at
  small computational and memory overhead, cf. http://arxiv.org/abs/1606.00915.
  Args:
    inputs: A tensor of size [batch, height_in, width_in, channels].
    blocks: A list of length equal to the number of ResNet blocks. Each element
      is a resnet_utils.Block object describing the units in the block.
    num_classes: Number of predicted classes for classification tasks.
      If 0 or None, we return the features before the logit layer.
    is_training: whether batch_norm layers are in training mode. If this is set
      to None, the callers can specify slim.batch_norm's is_training parameter
      from an outer slim.arg_scope.
    global_pool: If True, we perform global average pooling before computing the
      logits. Set to True for image classification, False for dense prediction.
    output_stride: If None, then the output will be computed at the nominal
      network stride. If output_stride is not None, it specifies the requested
      ratio of input to output spatial resolution.
    include_root_block: If True, include the initial convolution followed by
      max-pooling, if False excludes it.
    spatial_squeeze: if True, logits is of shape [B, C], if false logits is
        of shape [B, 1, 1, C], where B is batch_size and C is number of classes.
        To use this parameter, the input images must be smaller than 300x300
        pixels, in which case the output logit layer does not contain spatial
        information and can be removed.
    store_non_strided_activations: If True, we compute non-strided (undecimated)
      activations at the last unit of each block and store them in the
      `outputs_collections` before subsampling them. This gives us access to
      higher resolution intermediate activations which are useful in some
      dense prediction problems but increases 4x the computation and memory cost
      at the last unit of each block.
    reuse: whether or not the network and its variables should be reused. To be
      able to reuse 'scope' must be given.
    scope: Optional variable_scope.
  Returns:
    net: A rank-4 tensor of size [batch, height_out, width_out, channels_out].
      If global_pool is False, then height_out and width_out are reduced by a
      factor of output_stride compared to the respective height_in and width_in,
      else both height_out and width_out equal one. If num_classes is 0 or None,
      then net is the output of the last ResNet block, potentially after global
      average pooling. If num_classes a non-zero integer, net contains the
      pre-softmax activations.
    end_points: A dictionary from components of the network to the corresponding
      activation.
  Raises:
    ValueError: If the target output_stride is not valid.
  """
  with tf.variable_scope(scope, 'resnet_v1', [inputs], reuse=reuse) as sc:
    end_points_collection = sc.original_name_scope + '_end_points'
    with slim.arg_scope([slim.conv2d, bottleneck,
                         resnet_utils.stack_blocks_dense],
                        outputs_collections=end_points_collection):
      with (slim.arg_scope([slim.batch_norm], is_training=is_training)
            if is_training is not None else NoOpScope()):
        net = inputs
        if include_root_block:
          if output_stride is not None:
            if output_stride % 4 != 0:
              raise ValueError('The output_stride needs to be a multiple of 4.')
            output_stride /= 4
          net = resnet_utils.conv2d_same(net, 64, 7, stride=2, scope='conv1')
          net = slim.max_pool2d(net, [3, 3], stride=2, scope='pool1')
        net = resnet_utils.stack_blocks_dense(net, blocks, output_stride,
                                              store_non_strided_activations)
        # Convert end_points_collection into a dictionary of end_points.
        end_points = slim.utils.convert_collection_to_dict(
            end_points_collection)

        if global_pool:
          # Global average pooling.
          net = tf.reduce_mean(net, [1, 2], name='pool5', keep_dims=True)
          end_points['global_pool'] = net
        if num_classes:
          net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None,
                            normalizer_fn=None, scope='logits')
          end_points[sc.name + '/logits'] = net
          if spatial_squeeze:
            net = tf.squeeze(net, [1, 2], name='SpatialSqueeze')
            end_points[sc.name + '/spatial_squeeze'] = net
          end_points['predictions'] = slim.softmax(net, scope='predictions')
        return net, end_points
resnet_v1.default_image_size = 224


def resnet_v1_block(scope, base_depth, num_units, stride):
  """Helper function for creating a resnet_v1 bottleneck block.
  Args:
    scope: The scope of the block.
    base_depth: The depth of the bottleneck layer for each unit.
    num_units: The number of units in the block.
    stride: The stride of the block, implemented as a stride in the last unit.
      All other units have stride=1.
  Returns:
    A resnet_v1 bottleneck block.
  """
  return resnet_utils.Block(scope, bottleneck, [{
      'depth': base_depth * 4,
      'depth_bottleneck': base_depth,
      'stride': 1
  }] * (num_units - 1) + [{
      'depth': base_depth * 4,
      'depth_bottleneck': base_depth,
      'stride': stride
  }])


def resnet_v1_16(inputs,
                 num_classes=None,
                 is_training=True,
                 global_pool=True,
                 output_stride=None,
                 spatial_squeeze=True,
                 store_non_strided_activations=False,
                 min_base_depth=8,
                 depth_multiplier=1,
                 reuse=None,
                 scope='resnet_v1_16'):
  """ResNet-50 model of [1]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
      resnet_v1_block('block1', base_depth=depth_func(64), num_units=1,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=2,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=3,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=2,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_16.default_image_size = resnet_v1.default_image_size

def resnet_v1_16_reduced(inputs,
                 num_classes=None,
                 is_training=True,
                 global_pool=True,
                 output_stride=None,
                 spatial_squeeze=True,
                 store_non_strided_activations=False,
                 min_base_depth=8,
                 depth_multiplier=1,
                 reuse=None,
                 scope='resnet_v1_16_reduced'):
  """ResNet-50 model of [1]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
      resnet_v1_block('block1', base_depth=depth_func(64), num_units=1,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=2,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=2,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=2,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_16_reduced.default_image_size = resnet_v1.default_image_size

def resnet_v1_16_reduced2(inputs,
                 num_classes=None,
                 is_training=True,
                 global_pool=True,
                 output_stride=None,
                 spatial_squeeze=True,
                 store_non_strided_activations=False,
                 min_base_depth=8,
                 depth_multiplier=1,
                 reuse=None,
                 scope='resnet_v1_16_reduced2'):
  """ResNet-50 model of [1]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
     resnet_v1_block('block1', base_depth=depth_func(64), num_units=1,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=2,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=1,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=2,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_16_reduced2.default_image_size = resnet_v1.default_image_size

def resnet_v1_16_reduced3(inputs,
                 num_classes=None,
                 is_training=True,
                 global_pool=True,
                 output_stride=None,
                 spatial_squeeze=True,
                 store_non_strided_activations=False,
                 min_base_depth=8,
                 depth_multiplier=1,
                 reuse=None,
                 scope='resnet_v1_16_reduced3'):
  """ResNet-50 model of [1]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
     resnet_v1_block('block1', base_depth=depth_func(64), num_units=1,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=1,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=1,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=2,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_16_reduced3.default_image_size = resnet_v1.default_image_size

def resnet_v1_16_reduced4(inputs,
                 num_classes=None,
                 is_training=True,
                 global_pool=True,
                 output_stride=None,
                 spatial_squeeze=True,
                 store_non_strided_activations=False,
                 min_base_depth=8,
                 depth_multiplier=1,
                 reuse=None,
                 scope='resnet_v1_16_reduced4'):
  """ResNet-50 model of [1]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
     resnet_v1_block('block1', base_depth=depth_func(64), num_units=1,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=1,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=1,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=1,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_16_reduced4.default_image_size = resnet_v1.default_image_size

def resnet_v1_101(inputs,
                  num_classes=None,
                  is_training=True,
                  global_pool=True,
                  output_stride=None,
                  spatial_squeeze=True,
                  store_non_strided_activations=False,
                  min_base_depth=8,
                  depth_multiplier=1,
                  reuse=None,
                  scope='resnet_v1_101'):
  """ResNet-101 model of [1]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
      resnet_v1_block('block1', base_depth=depth_func(64), num_units=3,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=4,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=23,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=3,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_101.default_image_size = resnet_v1.default_image_size


def resnet_v1_152(inputs,
                  num_classes=None,
                  is_training=True,
                  global_pool=True,
                  output_stride=None,
                  store_non_strided_activations=False,
                  spatial_squeeze=True,
                  min_base_depth=8,
                  depth_multiplier=1,
                  reuse=None,
                  scope='resnet_v1_152'):
  """ResNet-152 model of [1]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
      resnet_v1_block('block1', base_depth=depth_func(64), num_units=3,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=8,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=36,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=3,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_152.default_image_size = resnet_v1.default_image_size


def resnet_v1_200(inputs,
                  num_classes=None,
                  is_training=True,
                  global_pool=True,
                  output_stride=None,
                  store_non_strided_activations=False,
                  spatial_squeeze=True,
                  min_base_depth=8,
                  depth_multiplier=1,
                  reuse=None,
                  scope='resnet_v1_200'):
  """ResNet-200 model of [2]. See resnet_v1() for arg and return description."""
  depth_func = lambda d: max(int(d * depth_multiplier), min_base_depth)
  blocks = [
      resnet_v1_block('block1', base_depth=depth_func(64), num_units=3,
                      stride=2),
      resnet_v1_block('block2', base_depth=depth_func(128), num_units=24,
                      stride=2),
      resnet_v1_block('block3', base_depth=depth_func(256), num_units=36,
                      stride=2),
      resnet_v1_block('block4', base_depth=depth_func(512), num_units=3,
                      stride=1),
  ]
  return resnet_v1(inputs, blocks, num_classes, is_training,
                   global_pool=global_pool, output_stride=output_stride,
                   include_root_block=True, spatial_squeeze=spatial_squeeze,
                   store_non_strided_activations=store_non_strided_activations,
                   reuse=reuse, scope=scope)
resnet_v1_200.default_image_size = resnet_v1.default_image_size

In [4]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)

In [5]:
from cleverhans.dataset import MNIST

tf.reset_default_graph()

tf.set_random_seed(1234)
sess = tf.Session()

train_start = 0
train_end = 1000
test_start = 1001
test_end = 1200

mnist = MNIST(train_start=train_start, train_end=train_end,
                test_start=test_start, test_end=test_end)
x_train, y_train = mnist.get_set('train')
x_test, y_test = mnist.get_set('test')




In [6]:
img_rows, img_cols, nchannels = x_train.shape[1:4]

In [7]:
print(img_rows)
print(img_cols)
print(nchannels)

# pip install --upgrade tensorflow==

28
28
1


In [8]:
x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y = tf.placeholder(tf.float32, shape=(None, 2))

nb_classes = 2 #? Y_train.shape[1]
nb_filters = 64 #?
model = ResNet(scope = "model1", nb_classes = nb_classes, nb_filters = nb_filters)
logits = model.get_logits(x)
#logits, _ = resnet_v1_16(inputs=x,num_classes=2, reuse=tf.AUTO_REUSE)
# logits_list.append(logits)
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)

hello


Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
train_op = optimizer.minimize(loss)

In [10]:
print(x_train.shape)
dataset_size=x_train.shape[0]
indices = list(range(dataset_size))
random_seed = 100
# print(indices)
np.random.seed(random_seed)
np.random.shuffle(indices)
# print(indices)

(1000, 28, 28, 1)


In [11]:
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [12]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])


In [13]:
print(x_train.shape)

(1000, 28, 28, 1)


In [14]:
epoch = 100
batch_size = 64
iter_num = dataset_size//batch_size
# ind

select_index= np.array(indices)
odd_Array = np.array([0,1,0,1,0,1,0,1,0,1])
# even_Array = np.array([1,0,1,0,1,0,1,0,1,0])
odd_Matrix = np.repeat(odd_Array[ np.newaxis, :], batch_size, axis=0)
# print(odd_Matrix)

#print(x_train.shape)
for n in range(epoch):
    print(n)
    indices = list(range(dataset_size))
    random_seed = n
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
    for i in range(iter_num):
        
        select = indices[i*batch_size:i*batch_size+batch_size]
        #print(select)
        x_batch = x_train[select]
        y_batch = y_train[select]
#         print(y_batch)
        y_mul = odd_Matrix*y_batch
        y_one = np.sum(y_mul, axis=1).astype(int)
        y_two = np.zeros((batch_size, 2))
        
        y_two[np.arange(batch_size), y_one] = 1
        
#         print(y_one)
#         print(y_two)
    
#         print(y_two.shape)
#         print(x_batch.shape)
#         print(y_one.shape)
        sess.run(train_op, feed_dict={x: x_batch, y: y_two})


        
        

(1000, 28, 28, 1)
0
[993, 859, 298, 553, 672, 971, 27, 231, 306, 706, 496, 558, 784, 239, 578, 55, 906, 175, 14, 77, 31, 481, 310, 311, 883, 788, 45, 103, 760, 1, 823, 710, 614, 790, 408, 736, 957, 366, 918, 267, 230, 996, 635, 698, 251, 783, 819, 141, 316, 587, 331, 295, 262, 432, 862, 582, 272, 270, 987, 319, 569, 643, 142, 202]
[413, 196, 264, 531, 252, 576, 738, 299, 740, 247, 926, 412, 389, 796, 601, 654, 261, 456, 386, 982, 909, 693, 236, 501, 497, 874, 452, 494, 923, 279, 638, 485, 568, 108, 367, 644, 785, 873, 65, 902, 317, 636, 666, 789, 958, 214, 97, 583, 466, 523, 255, 385, 545, 382, 489, 513, 780, 904, 122, 946, 60, 844, 372, 283]
[939, 608, 495, 630, 596, 713, 753, 624, 312, 8, 529, 876, 101, 615, 520, 249, 278, 424, 384, 395, 54, 34, 271, 30, 403, 757, 975, 990, 457, 362, 860, 483, 580, 986, 688, 864, 927, 378, 320, 942, 351, 418, 945, 795, 145, 826, 492, 751, 240, 648, 886, 144, 767, 999, 467, 315, 989, 922, 717, 458, 773, 204, 834, 832]
[435, 5, 769, 679, 722, 215, 653,

[642, 545, 477, 547, 63, 413, 384, 889, 322, 451, 781, 234, 656, 557, 462, 424, 880, 303, 884, 689, 219, 870, 296, 21, 760, 420, 325, 199, 450, 137, 536, 696, 717, 646, 628, 212, 961, 934, 921, 505, 815, 231, 405, 729, 568, 317, 183, 856, 854, 693, 609, 626, 485, 278, 630, 20, 170, 401, 924, 566, 176, 327, 706, 198]
[470, 842, 694, 266, 615, 732, 492, 130, 338, 851, 489, 572, 606, 193, 140, 416, 660, 476, 714, 152, 849, 898, 10, 269, 96, 210, 984, 752, 569, 813, 775, 548, 943, 532, 933, 332, 75, 712, 77, 263, 993, 149, 932, 514, 940, 469, 564, 774, 461, 253, 771, 845, 369, 321, 695, 151, 949, 302, 190, 586, 348, 243, 793, 87]
[751, 655, 909, 778, 418, 288, 648, 166, 595, 937, 155, 356, 381, 279, 126, 707, 777, 22, 616, 665, 282, 903, 471, 738, 367, 25, 196, 64, 15, 466, 297, 883, 621, 336, 820, 26, 588, 43, 497, 792, 515, 955, 561, 917, 454, 387, 71, 542, 456, 633, 431, 627, 653, 728, 264, 209, 316, 896, 513, 313, 534, 829, 319, 7]
2
[37, 726, 846, 295, 924, 658, 682, 286, 880, 272, 13

[492, 166, 938, 821, 315, 361, 138, 276, 384, 568, 792, 88, 794, 280, 460, 617, 867, 599, 202, 402, 340, 365, 23, 345, 933, 334, 310, 899, 353, 953, 240, 318, 735, 262, 98, 740, 923, 865, 89, 515, 331, 678, 433, 913, 723, 486, 636, 832, 748, 260, 305, 925, 196, 140, 594, 122, 494, 110, 84, 717, 513, 482, 680, 922]
[74, 956, 778, 273, 396, 154, 191, 915, 187, 50, 38, 418, 876, 208, 375, 463, 470, 517, 75, 248, 559, 302, 582, 739, 995, 11, 973, 485, 462, 976, 627, 281, 253, 983, 990, 911, 577, 986, 219, 425, 369, 829, 69, 980, 580, 21, 77, 547, 993, 946, 125, 228, 42, 516, 572, 576, 801, 621, 374, 205, 969, 991, 6, 493]
[710, 943, 766, 736, 259, 282, 634, 590, 882, 974, 455, 320, 434, 663, 299, 436, 675, 812, 438, 325, 403, 265, 564, 602, 815, 278, 655, 759, 732, 711, 553, 751, 888, 896, 231, 245, 167, 502, 332, 807, 783, 426, 541, 162, 149, 226, 685, 94, 221, 597, 699, 473, 977, 601, 873, 34, 217, 72, 220, 213, 860, 394, 963, 367]
[12, 918, 343, 198, 725, 696, 234, 695, 420, 841, 989, 6

[272, 464, 240, 310, 53, 346, 823, 443, 887, 386, 946, 285, 338, 855, 232, 155, 278, 561, 994, 697, 908, 882, 798, 306, 273, 454, 62, 854, 639, 502, 452, 634, 579, 828, 111, 872, 876, 888, 787, 976, 733, 834, 610, 933, 432, 840, 940, 197, 687, 68, 545, 495, 652, 536, 417, 796, 409, 187, 396, 488, 892, 421, 330, 342]
[115, 556, 363, 313, 480, 481, 440, 194, 901, 895, 683, 224, 742, 51, 936, 242, 75, 219, 222, 317, 568, 816, 359, 319, 776, 369, 650, 858, 4, 282, 6, 738, 128, 248, 305, 448, 15, 504, 292, 662, 679, 93, 318, 303, 921, 613, 56, 383, 915, 357, 615, 966, 34, 552, 170, 466, 713, 875, 186, 10, 85, 658, 628, 325]
[785, 975, 189, 116, 880, 807, 692, 48, 894, 125, 46, 256, 587, 731, 680, 16, 642, 405, 689, 47, 157, 674, 999, 563, 382, 218, 843, 788, 90, 583, 953, 54, 673, 457, 893, 127, 109, 484, 43, 422, 651, 714, 771, 657, 986, 848, 145, 927, 890, 808, 631, 289, 824, 315, 581, 468, 98, 226, 450, 381, 775, 461, 903, 277]
[321, 586, 943, 483, 442, 503, 582, 654, 832, 576, 641, 907,

[680, 285, 127, 371, 411, 644, 981, 365, 600, 80, 158, 423, 747, 324, 865, 780, 675, 642, 626, 731, 284, 437, 179, 870, 839, 222, 122, 761, 992, 750, 873, 869, 447, 389, 431, 244, 384, 679, 220, 449, 15, 552, 523, 397, 395, 253, 934, 793, 846, 360, 990, 766, 172, 686, 147, 186, 43, 616, 528, 925, 296, 603, 138, 54]
[669, 51, 819, 318, 22, 500, 800, 294, 209, 712, 142, 50, 166, 621, 655, 81, 666, 983, 900, 66, 732, 656, 935, 988, 510, 513, 64, 734, 264, 864, 463, 498, 20, 367, 768, 696, 719, 578, 261, 652, 720, 806, 570, 459, 822, 188, 251, 71, 664, 418, 336, 328, 350, 909, 928, 986, 375, 557, 684, 451, 809, 505, 677, 833]
[526, 480, 446, 218, 239, 752, 525, 192, 795, 191, 508, 435, 132, 556, 252, 671, 901, 263, 70, 272, 507, 327, 115, 323, 957, 663, 866, 504, 620, 773, 702, 445, 619, 751, 609, 278, 785, 867, 2, 229, 419, 923, 577, 853, 199, 950, 515, 65, 955, 40, 742, 514, 420, 492, 562, 273, 694, 627, 682, 757, 177, 300, 37, 478]
[202, 236, 149, 217, 673, 564, 911, 796, 399, 302, 852,

...

[368, 890, 762, 984, 217, 937, 811, 112, 84, 269, 928, 311, 878, 99, 554, 341, 142, 177, 307, 790, 580, 819, 842, 140, 255, 841, 359, 225, 205, 871, 858, 783, 448, 394, 929, 950, 355, 602, 396, 838, 422, 493, 125, 593, 72, 146, 26, 533, 528, 333, 647, 429, 570, 495, 914, 780, 497, 437, 611, 449, 872, 344, 218, 572]
[77, 22, 809, 302, 659, 6, 694, 896, 420, 973, 188, 546, 296, 413, 517, 607, 735, 415, 173, 488, 91, 885, 742, 706, 769, 281, 45, 476, 549, 917, 121, 50, 844, 159, 583, 786, 214, 848, 717, 48, 166, 603, 191, 665, 988, 94, 247, 264, 658, 273, 257, 10, 137, 85, 1, 932, 698, 573, 70, 918, 251, 492, 194, 697]
[375, 958, 133, 365, 463, 727, 161, 74, 883, 114, 891, 304, 862, 262, 200, 138, 855, 806, 236, 174, 318, 364, 57, 328, 610, 867, 399, 479, 63, 0, 992, 295, 754, 180, 118, 544, 605, 796, 39, 770, 761, 640, 726, 522, 215, 931, 886, 724, 683, 826, 451, 322, 962, 239, 268, 53, 110, 330, 83, 8, 974, 847, 398, 78]
92
[92, 209, 817, 130, 995, 87, 666, 359, 779, 157, 613, 185, 994,

[138, 284, 17, 800, 693, 275, 40, 597, 850, 646, 519, 250, 968, 22, 372, 838, 604, 828, 504, 639, 779, 403, 380, 705, 699, 826, 244, 144, 220, 311, 122, 866, 417, 292, 833, 963, 215, 794, 235, 780, 110, 338, 555, 868, 625, 854, 374, 53, 169, 424, 567, 741, 732, 862, 488, 155, 81, 11, 38, 493, 808, 396, 977, 461]
[523, 634, 722, 851, 86, 635, 671, 751, 675, 140, 704, 495, 377, 892, 58, 286, 819, 522, 656, 127, 14, 31, 535, 413, 346, 411, 572, 548, 70, 678, 906, 245, 401, 409, 505, 903, 650, 465, 713, 923, 172, 547, 283, 162, 134, 647, 853, 669, 362, 596, 606, 332, 590, 865, 238, 76, 744, 136, 80, 192, 142, 925, 487, 103]
[360, 799, 265, 912, 872, 491, 618, 8, 935, 883, 711, 366, 446, 239, 217, 349, 660, 34, 569, 462, 608, 443, 655, 368, 932, 967, 926, 412, 510, 858, 802, 251, 306, 173, 975, 45, 3, 528, 148, 297, 757, 503, 702, 712, 891, 979, 938, 529, 189, 291, 998, 746, 320, 659, 197, 419, 549, 792, 90, 365, 59, 717, 775, 202]
[344, 352, 176, 398, 632, 252, 580, 937, 924, 339, 620, 857

[364, 207, 775, 439, 393, 426, 141, 963, 83, 232, 683, 99, 743, 10, 536, 539, 492, 582, 205, 589, 868, 837, 200, 892, 402, 138, 597, 814, 380, 763, 675, 904, 787, 624, 395, 401, 975, 467, 250, 122, 995, 636, 258, 115, 929, 43, 952, 421, 960, 701, 431, 861, 174, 556, 595, 44, 676, 849, 179, 243, 309, 177, 954, 618]
[291, 192, 936, 566, 809, 984, 422, 437, 110, 698, 459, 690, 397, 218, 847, 723, 5, 652, 633, 212, 817, 440, 687, 79, 22, 734, 98, 703, 199, 176, 254, 90, 396, 768, 993, 500, 753, 166, 351, 14, 663, 153, 151, 86, 286, 339, 164, 848, 408, 419, 887, 608, 173, 64, 956, 114, 253, 233, 267, 70, 134, 846, 629, 759]
[913, 150, 991, 188, 533, 989, 869, 825, 798, 986, 688, 812, 388, 819, 579, 169, 902, 146, 403, 548, 631, 13, 259, 873, 779, 208, 737, 540, 281, 310, 835, 336, 713, 557, 423, 264, 29, 175, 304, 194, 756, 390, 144, 635, 645, 781, 170, 85, 875, 732, 796, 216, 969, 564, 730, 145, 864, 65, 891, 270, 181, 384, 517, 7]
[521, 532, 786, 654, 534, 706, 222, 939, 808, 399, 733, 70

[697, 262, 931, 115, 37, 204, 833, 369, 363, 324, 756, 454, 763, 555, 533, 623, 827, 430, 493, 426, 253, 30, 100, 275, 390, 741, 197, 323, 784, 712, 888, 733, 635, 459, 864, 55, 330, 954, 935, 950, 828, 10, 295, 684, 298, 469, 698, 525, 175, 12, 209, 287, 279, 488, 749, 871, 948, 991, 985, 634, 66, 1, 607, 811]
[945, 927, 609, 721, 193, 42, 514, 124, 745, 386, 70, 570, 46, 252, 676, 736, 964, 110, 787, 153, 530, 976, 150, 734, 17, 549, 755, 404, 989, 867, 422, 992, 29, 802, 119, 645, 40, 596, 325, 513, 351, 902, 0, 744, 674, 412, 879, 522, 563, 191, 216, 559, 242, 969, 605, 913, 773, 750, 856, 586, 509, 223, 59, 653]
[147, 492, 439, 952, 823, 899, 394, 471, 174, 219, 134, 272, 192, 212, 692, 805, 345, 813, 941, 20, 251, 173, 739, 85, 136, 999, 720, 329, 121, 966, 576, 978, 246, 176, 468, 839, 705, 74, 577, 528, 546, 911, 618, 748, 19, 807, 723, 986, 834, 269, 387, 448, 846, 840, 924, 621, 693, 205, 806, 356, 84, 353, 870, 665]
[393, 914, 23, 820, 690, 502, 796, 437, 217, 128, 944, 700,

99
[890, 983, 107, 609, 113, 350, 22, 136, 674, 46, 436, 437, 813, 271, 563, 214, 630, 294, 245, 746, 435, 455, 954, 109, 50, 755, 344, 911, 526, 610, 900, 518, 668, 720, 782, 571, 86, 129, 512, 124, 267, 941, 840, 701, 582, 931, 958, 434, 283, 132, 839, 269, 683, 407, 396, 191, 597, 499, 972, 949, 452, 3, 177, 386]
[693, 100, 978, 48, 495, 814, 698, 709, 57, 489, 131, 126, 821, 706, 786, 749, 841, 697, 548, 719, 473, 864, 584, 459, 515, 774, 809, 84, 580, 493, 305, 681, 63, 152, 498, 618, 594, 509, 520, 172, 292, 456, 481, 138, 219, 742, 451, 685, 539, 704, 74, 421, 637, 781, 569, 639, 299, 651, 81, 838, 832, 115, 477, 370]
[560, 588, 926, 154, 775, 923, 348, 266, 360, 467, 574, 558, 943, 6, 307, 598, 373, 903, 40, 405, 552, 300, 17, 167, 976, 816, 522, 478, 678, 659, 743, 18, 32, 53, 377, 963, 684, 397, 272, 258, 896, 78, 199, 636, 42, 718, 957, 27, 262, 308, 25, 368, 899, 230, 796, 465, 208, 134, 105, 7, 253, 295, 225, 959]
[752, 686, 708, 780, 561, 544, 846, 669, 211, 119, 384, 80,

In [16]:
saver = tf.train.Saver(max_to_keep=10, keep_checkpoint_every_n_hours=0.5)
saver.save(sess, 'res_50_model.ckpt')

'res_50_model.ckpt'

In [17]:
batch_size_val = 199
odd_Matrix_2 = np.repeat(odd_Array[ np.newaxis, :], batch_size_val, axis=0)
y_temp = odd_Matrix_2*y_test
y_temp_one = np.sum(y_temp, axis=1).astype(int)
y_val = np.zeros((batch_size_val, 2))
y_val[np.arange(batch_size_val), y_temp_one] = 1

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_val, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: x_test, y: y_val}))

0.9296482


In [18]:
from cleverhans.train import train
from cleverhans.utils_tf import model_eval
from cleverhans.attacks import CarliniWagnerL2
from cleverhans.model_zoo.alexNet import AlexNet
import os

In [19]:
VIZ_ENABLED = True
BATCH_SIZE = 128
NB_EPOCHS = 6
SOURCE_SAMPLES = 100
LEARNING_RATE = .001
CW_LEARNING_RATE = .2
ATTACK_ITERATIONS = 100
MODEL_PATH = os.path.join('models', 'mnist')
TARGETED = True
source_samples = SOURCE_SAMPLES
attack_iterations = ATTACK_ITERATIONS

In [20]:
train_params = {
  'nb_epochs': 1,
  'batch_size': 128,
  'learning_rate': 0.001,
  'filename': os.path.split(MODEL_PATH)[-1]
}
rng = np.random.RandomState([2017, 8, 30])

In [21]:

cw = CarliniWagnerL2(model, sess=sess)
saver = tf.train.Saver()
saver.save(sess, MODEL_PATH)

D:\GithubRepo\Engineering-and-Debugging-Tools-for-Deep-Neural-Networks-\cleverhans\attacks_tf.py:27: UserWarning: attacks_tf is deprecated and will be removed on 2019-07-18 or after. Code should import functions from their new locations directly.
  warnings.warn("attacks_tf is deprecated and will be removed on 2019-07-18"


'models\\mnist'

In [22]:
adv_inputs = np.array(
          [[instance] * nb_classes for
           instance in x_test[:source_samples]], dtype=np.float32)
one_hot = np.zeros((nb_classes, nb_classes))
one_hot[np.arange(nb_classes), np.arange(nb_classes)] = 1
adv_inputs = adv_inputs.reshape(
    (source_samples * nb_classes, img_rows, img_cols, nchannels))
adv_ys = np.array([one_hot] * source_samples,
                  dtype=np.float32).reshape((source_samples *
                                             nb_classes, nb_classes))
yname = "y_target"
cw_params_batch_size = source_samples * nb_classes
cw_params = {'binary_search_steps': 1,
             yname: adv_ys,
             'max_iterations': attack_iterations,
             'learning_rate': CW_LEARNING_RATE,
             'batch_size': cw_params_batch_size,
             'initial_const': 10}

In [23]:
adv = cw.generate_np(adv_inputs,
                     **cw_params)
x_clean = np.array(
          [[instance] * nb_classes for
           instance in x_test[:source_samples]], dtype=np.float32)
x_clean = x_clean.reshape(
    (source_samples * nb_classes, img_rows, img_cols, nchannels))
avg_perturbation = np.mean( np.abs(adv-x_clean))

# print(adv.shape)
# print(adv)
print(avg_perturbation)

[INFO 2019-12-10 20:44:22,500 cleverhans] Constructing new graph for attack CarliniWagnerL2



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


D:\GithubRepo\Engineering-and-Debugging-Tools-for-Deep-Neural-Networks-\cleverhans\compat.py:22: UserWarning: <function reduce_sum_v1 at 0x000002CC523DCAE8> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
D:\GithubRepo\Engineering-and-Debugging-Tools-for-Deep-Neural-Networks-\cleverhans\compat.py:22: UserWarning: <function reduce_max_v1 at 0x000002CC523E09D8> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "



Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
0.009473813


In [23]:
diff_reshape = np.abs(adv-x_clean).reshape(-1, 28*28)
diff_sum = np.sum(diff_reshape, axis=1)
avg_perturbation = np.mean(diff_sum)
print(avg_perturbation)

7.612117
